In [9]:
#Sort Different columns: 
import pandas as pd 
import numpy as np
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit_aer.primitives import Sampler
import re
import openpyxl

df = pd.read_csv('Files/business-financial-data-december-2024-quarter-csv.csv')
type = detectColumns(df,['Suppressed'])
print(type)    

0
{'Suppressed': {'0': 89, '1': 10}}


In [ ]:
import pandas as pd
import numpy as np
def digitSorting(df,col,asc):
    df[col] = df[col].astype(str)
    df = df.sort_values(by=col,ascending=asc, ignore_index=True)
    return df

df = pd.read_csv("Files/student_dataset.csv")
df = digitSorting(df,'Grade',asc=False)
print(df)

In [ ]:
def multiIndex(dataFrame, colToCheck, colType, yearOnly = False, asc=True):
    # Step 1: Get last indices of each year
    if yearOnly == True:
        yearsWithLastIndex = get_last_indices_of_each_year(dataFrame[colToCheck], True)
    else: 
        if (colType['0'] == 30 or  colType['1'] == 70) or (colType['0'] == 20 or  colType['1'] == 80):
            yearsWithLastIndex = get_last_indices_of_each_year(pd.to_datetime(dataFrame[colToCheck]),False)
        if(colType['1']==30 or colType['0']==70) or (colType['1'] ==50 or colType['0']==50):
            yearsWithLastIndex = get_last_indices_of_each_year(dataFrame[colToCheck],False,True,asc=asc)
            asc=True
        else:
            yearsWithLastIndex = get_last_indices_of_each_year(dataFrame[colToCheck], False)
    if asc==False:
        yearsWithLastIndex = dict(reversed(list(yearsWithLastIndex.items())))
        
    nameOfGroups = list(yearsWithLastIndex.keys())
    last_indices = list(yearsWithLastIndex.values())

    # Step 2: Compute counts from last indices
    group_sizes = []
    prev = -1
    for idx in last_indices:
        group_sizes.append(idx - prev)
        prev = idx
    print(group_sizes)
    # Step 3: Create array per group
    objOfGroups = {
        f'key{i}': np.array([f'Group of {year}'] * group_sizes[i]) for i, year in enumerate(nameOfGroups)
    }
    
    # Step 4: Combine into one array
    outside = np.concatenate(list(objOfGroups.values()))
    inside = np.arange(len(outside))
        
    print(outside)
    # Step 5: Create inside index
    
    multi_index = pd.MultiIndex.from_arrays([outside, inside], names=["Group", "Sr No."])
    
    dataFrame = dataFrame.reindex(range(len(multi_index)))
    dataFrame.set_index(multi_index,inplace=True,)
    print("Multicalled")
    return dataFrame

def get_last_indices_of_each_year(date_series, YearOnly=False,rol=False, a=0.60,asc=True):
    
    # data_series = data_series.apply(pd.to_numeric, errors='coerce').astype('Int64')
    if rol==True and findDuplicate(date_series).count() <=8:
            df = pd.DataFrame({'year':date_series},index=np.arange(len(date_series))).astype(str)
            df['half'] = df['year'].apply(lambda x: x[:round(len(str(x))*a)])
            print(a)
            last_indices =  df.groupby('half',).apply(lambda x: x.index[-1]).to_dict()
            if asc ==False:
                last_indices = dict(reversed(list(last_indices.items())))
                
            group_sizes = []
            prev = -1
            k=0
            for i,idx in enumerate(last_indices.values()):
                group_sizes.append(idx - prev)
                prev = idx
                if group_sizes[i]<= 15:
                    k=k+1
            if int(len(last_indices)*0.40)<=k and a>=0.10:
                last_indices = get_last_indices_of_each_year(date_series,False,True,a=a-0.10, asc=asc)
            return last_indices
    if YearOnly == True:
        print("yearOnly work")
        df = pd.DataFrame({'year': date_series}, index=np.arange(len(date_series)))
    
    else:
        # Extract year
        try:
            years = date_series.dt.year
            # Create a DataFrame with index
            df = pd.DataFrame({'year': years}, index=np.arange(len(date_series)))
            print("Year Extracted")
        except:
            #Create a DataFrame with index
            df = pd.DataFrame({'year': date_series}, index=np.arange(len(date_series)))
        
    # Get last index of each year group
    last_indices = df.groupby('year').apply(lambda x: x.index[-1]).to_dict()
        
    print("last index called")
    return last_indices


In [ ]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def sortRollCol(df, col,asc):
    digits_df = df[col].astype(str).apply(lambda x: pd.Series(list(x)))
    
    digits_df['original_index'] = df.index
    
    sorted_digits_df = recursiveSort(digits_df)

    sorted_indices = sorted_digits_df['original_index'].values
    if asc == False:
        sorted_indices = reversed(sorted_indices)
        
    sorted_df = df.loc[sorted_indices].reset_index(drop=True)

    return sorted_df


def recursiveSort(df_digits, col=0):
    if col >= int(len(df_digits.columns) - 1):  # exclude 'original_index' column
        return df_digits

    # Sort by the current digit column
    df_digits = df_digits.sort_values(by=col, kind='stable', ignore_index=True)

    # Group by current digit and recursively sort each group
    result = []
    for value, group in df_digits.groupby(col, sort=False):
        sorted_group = recursiveSort(group.reset_index(drop=True), col + 1)
        result.append(sorted_group)
    
    return pd.concat(result, ignore_index=True)


# Example usage
df = pd.read_csv('Files/student_dataset.csv')  # Your dataset with 'Roll No.'
asc = False
df = sortRollCol(df,'Roll No.',asc=asc)
type = {
        '0' : 70, 
        '1' : 30
        }
df = multiIndex(df,colToCheck='Roll No.',colType=type,asc=asc)
df

In [ ]:
def dataClean(df,colTypes):
    for i in df.index[(df.isna().sum(axis=1)==len(df.columns))]:
        df.drop(index=i, inplace=True)
        
    for i in colTypes.keys():
        if df[i].isna().sum() >=len(df[i])-2:
            df.drop(columns=[i],inplace=True)
            continue
        
        if df[i].isna().sum() != 0:#IF column has no type
                print(i)
                if (colTypes[i]['0'] == 60 or colTypes[i]['1'] == 40): # if column is type of year only
                    df[i] = df[i].fillna(2100)
                elif colTypes[i]['0'] == 30 or  colTypes[i]['1'] == 70:#if column is type of date only 
                    df[i] = df[i].fillna(pd.to_datetime('2030-01-01'))
                elif colTypes[i]['0'] == 20 or  colTypes[i]['1'] == 80 :#if column is type of date and time 
                    df[i]= df[i].fillna('2030-01-01')
                elif colTypes[i]['0'] == 70 or colTypes[i]['1']==30: #Roll no type
                    pass
                elif colTypes[i]['1']==50 or colTypes[i]['0']==50:#if it is of type id 
                    df[i]=df[i].fillna("Unknown")
                elif colTypes[i]['0']==90 or colTypes[i]['1'] ==10:#if it of type oneor2digit
                    df[i] = df[i].fillna(0)
                elif colTypes[i]['0'] == 80 or  colTypes[i]['1'] == 20 : #if it type of nemric
                    df[i]=df[i].fillna(0.0)
                elif colTypes[i]['0'] == 40 or  colTypes[i]['1'] == 60 : #string or object
                    df[i] = df[i].fillna('NAN')
        continue
    return df

df = pd.read_csv('Files/business-financial-data-december-2024-quarter-csv.csv')
col = detectColumns(df,df.columns)
print(df.isna().sum())
df = dataClean(df,col)
print(df.isna().sum())


In [ ]:
def findDuplicate(series):
    series = series[series.duplicated(keep=False)]
    return series

df = pd.read_csv('Files/twitter_training.csv')
dup = findDuplicate(df['Comments'])
print(dup)


In [8]:
df = pd.read_csv('Files/employee_time_log.csv') 
df = kmeansOnString(df,'Employee_ID')
print(df)

time By Model  44.38958549499512
Time in Normalizig : 0.003229379653930664


KeyboardInterrupt: 

In [26]:
x = ['bdca.930-y','bdca.uei-9','ueii/783-77']
df = pd.DataFrame(x)
for i,v in enumerate(df[0]):
    df[0][i] = df[0][i][:df[0][i].find('.')]+df[0][i][df[0][i].find('.')+1:]
    
df

C:\Users\tikes\AppData\Local\Temp\ipykernel_15320\3824584049.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[0][i] = df[0][i][:df[0][i].find('.')]+df[0][i][df[0][i].find('.')+1:]


,0
0,bdca930-y
1,bdcauei-9
2,ueii/783-7ueii/783-77


In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd
import time
def kmeansOnString(df,col):
    phrase = (df[col]).astype(str)
    # Step 1: Convert strings to embeddings using a Sentence Transformer (BERT-based model)
    st = time.time()
    model = SentenceTransformer('all-MiniLM-L6-v2')  # a lightweight, efficient model
    embeddings = model.encode(phrase)
    et = time.time()
    print("time By Model ", et-st)
    
    st = et
    # Optionally, normalize embeddings if needed:
    from sklearn.preprocessing import normalize
    embeddings = normalize(embeddings)
    et = time.time()
    print(f"Time in Normalizig : {et-st}")
    
    st = et
    # Step 2: Apply KMeans clustering on these embeddings
    # Using n_init and max_iter helps ensure stable convergence.
    k =  int(np.ceil(len(df[col]) / 80))# number of clusters
    kmeans = KMeans(n_clusters=7, n_init=19, max_iter=450, random_state=42)
    df['Cluster Id'] = kmeans.fit_predict(embeddings)
    et = time.time()
    print(f"Time clustering : {et-st}")
    
    st = et
    # Step 3: Evaluate the clustering with Silhouette Score
    score = silhouette_score(embeddings, df['Cluster Id'])
    print("Silhouette Score:", score)
    et = time.time()
    print(f"Time in Normalizig : {et-st}")
    
    df = df.sort_values(by='Cluster Id').drop(columns=['Cluster Id'])
    return df

In [6]:
from qiskit import QuantumCircuit
import pandas as pd
import math
def detectColumns(df, prioColumns):
    result = {}    
    for col in prioColumns:
        qc =  QuantumCircuit(1,1)
        col_data = df[col]
        col_str = df[col].astype(str).str.strip()
        # 3. Check for Date values (type 2)
        if check_date_format(col_str):
            p = 0.70000
                
        # 4. Check for DateTime values (type 3)
        elif check_datetime_format(col_str):
            p = 0.800000
            
        elif OneOr2digitDetection(col_data):
            p = 0.100000
        # 1. Check for Roll Numbers (type 4)
        elif pd.api.types.is_numeric_dtype(col_data):
            p = 0.2000
            # 2. Check for Year values (type 1)
            if check_year_values(col_data):
                p = 0.40000
            elif check_roll_number(col_data):
                p = 0.30000

        elif pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_object_dtype(df[col]):
            p = 0.600000
            if detectIdTypeCol(col_data):
                p = 0.5
        
        angle = 2 * math.asin(math.sqrt(p))
        qc.ry(angle, 0)
        # Initialize result as 0 (unrecognized type)
        result[col] = measurCir(qc,0)
        
    return result

def check_roll_number(col_data):
    try:
        # Convert numbers to strings for checking patterns
        sample_start = [str(i) for i in col_data.head(10)]
        sample_middle = [str(i) for i in col_data.iloc[int(len(col_data)/2)-5:int(len(col_data)/2)+5]]
        sample_end = [str(i) for i in col_data.iloc[-10:]]
        
        # Combine samples
        samples = sample_start + sample_middle + sample_end
        
        # Check if all numbers have the same length and >= 5 digits
        if len(set(len(str(x)) for x in samples)) == 1:
            length = len(str(samples[0]))
            if length >= 5:
                # Check if all numbers start with the same digit
                first_digits = str(samples[0])[0]
                return all(str(x).startswith(first_digit) for x in samples)
    except:
        pass
    return False

def check_year_values(col_data):
    if pd.api.types.is_string_dtype(col_data):
        try:
            col_data = pd.to_numeric(col_data)
        except:
            pass
    # Handle if column is numeric and looks like a year
    if pd.api.types.is_numeric_dtype(col_data) or  pd.api.types.is_float_dtype(col_data) or  pd.api.types.is_integer_dtype(col_data):
        if col_data.dropna().empty == False:
            if pd.api.types.is_float_dtype(col_data):
            # Check if float values have only 2 decimal places
                if col_data.dropna().apply(lambda x: round(x, 2) == x).all():
                    if col_data.dropna().between(1800, 2050).all():
                        return True # Only year
            # For integer values
            elif col_data.dropna().between(1800, 2100).all():
                True # Only year
                
    return False

def check_date_format(col_str):
    # Check for common date formats (yyyy-mm-dd, dd-mm-yyyy, etc.)
    date_pattern = r'^\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}$'
    return col_str.str.match(date_pattern).all()

def check_datetime_format(col_str):
    # Check for datetime format (date + time)
    datetime_pattern = r'\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}.*\d{1,2}:\d{2}'
    return col_str.str.contains(datetime_pattern).all()

def detectIdTypeCol(col_data):
    if pd.api.types.is_string_dtype(col_data):
        pattern = r'\b[A-Z0-9]{1,4}[-_./]?[A-Z0-9]{2,6}[-_./]?[A-Z0-9]{0,5}\b'
        return all(re.fullmatch(pattern, item) for item in col_data)
    
def OneOr2digitDetection(col_data):
    non2Digit=0
    for i,v  in enumerate(col_data):
        if len(str(v)) > 2 and str(v)!='nan':
            non2Digit=non2Digit+1
    print(non2Digit)
    if len(col_data)/2.2 > non2Digit:
        return True
    return False

def detectSimpleDtypes(col_data):
    if pd.api.types.is_integer_dtype(col_data):
        return 'allInt'
    if pd.api.types.is_float_dtype(col_data):
        if col_data.isna().all()== False:
            return 'numaric'
    if pd.api.types.is_string_dtype(col_data) or pd.api.types.is_object_dtype(col_data):
        return 'str'
    return None

def clusterDateTimeCol(fContent, col,no,ascending=True):
    if no ==1:
        # Try to detect and sort if the column is just year values
        fContent = fContent.sort_values(by=col,ignore_index=True, ascending=ascending)
        fContent = fContent.reset_index(drop=True)
    elif no == 2 or 3:
        # Now, try to detect proper date/datetime columns
        try:
            # Avoid processing numeric-only or zero-filled columns
            sample_vals = fContent[col].astype(str).str.strip().replace('0', np.nan).dropna()
            if len(sample_vals) == 0:
                return fContent# All values are zero or empty-like
            
            # Try parsing
            try:
                parsed_col = pd.to_datetime(fContent[col], errors='raise')
            except:
                parsed_col = pd.to_datetime(fContent[col], dayfirst=True, errors='raise')
               
            if all(parsed_col.dt.time == pd.to_datetime('00:00:00').time()) and no == 2:  # Only date
                fContent[col] = parsed_col
                fContent = clean_and_sort_date_column(fContent, col, ascending)
                fContent = fContent.reset_index(drop=True)
                # Replace original column with parsed datetime values
            elif no == 3:  # Date + time
                fContent[col] = parsed_col
                fContent = handle_datetime_column(fContent, col, ascending)
                fContent = fContent.reset_index(drop=True)
        except Exception as e:
            return fContent # Not a datetime column
        
    return fContent 

def clean_and_sort_date_column(dff, column_name, ascending=True):
        try:
            
            # Step 2: Drop NaT (invalid formats)
            dff = dff.dropna(subset=[column_name])
            
            # Step 3: Sort the DataFrame by that column
            dff = dff.sort_values(by=column_name, ascending=ascending)

            # Optional: Format to clean date string (YYYY-MM-DD)
            dff[column_name] = dff[column_name].dt.strftime('%Y-%m-%d')
            print(f"{column_name} date called ")
            return dff
        
        except Exception as e:
            print(f"⚠️ Error while processing date column: {e}")
            return dff

def handle_datetime_column(df, column_name, ascending):
    print(f"{column_name} dateTime")
    # Check if most values in column are datetime with time
    values = df[column_name].dropna().astype(str).head(20)
    count_datetime = sum([pd.api.is_datetime(v) for v in values])

    if count_datetime >= len(values) // 2:  # At least half must be datetime-like
        # Convert full column to datetime
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        # Drop rows with invalid dates
        df = df.dropna(subset=[column_name])
        # Sort by that column
        df = df.sort_values(by=column_name,ascending=ascending).reset_index(drop=True)
        print(f"[INFO] '{column_name}' successfully recognized and sorted as datetime.")
    else:
        print(f"[INFO] '{column_name}' does not contain proper datetime with time.")

    return df

In [5]:
def measurCir(qc,q_num):
    qc.measure(q_num,q_num)
    simulator = AerSimulator()
    # Transpile & run
    compiled = transpile(qc, simulator)
    r = simulator.run(compiled, shots=100000).result()
    counts = r.get_counts()
    for k,v in counts.items():
        counts[k] = int(v/1000)
    return counts